In [130]:
import math
import time
import random
import sympy
import warnings
from random import randint, seed
import sys
from ecpy.curves import Curve,Point
from Crypto.Hash import SHA3_256, HMAC, SHA256
import requests
from Crypto.Cipher import AES
from Crypto import Random
from Crypto.Util.Padding import pad
from Crypto.Util.Padding import unpad
import random
import re
import json

API_URL = 'http://harpoon1.sabanciuniv.edu:9999/'

stuID = 28315
stuIDB = 18007

#Server's Identitiy public key
IKey_Ser = Point(0x1d42d0b0e55ccba0dd86df9f32f44c4efd7cbcdbbb7f36fd38b2ca680ab126e9,
                 0xce091928fa3738dc18f529bf269ade830eeb78672244fd2bdfbadcb26c4894ff, Curve.get_curve('secp256k1'))

def IKRegReq(h,s,x,y):
    mes = {'ID':stuID, 'H': h, 'S': s, 'IKPUB.X': x, 'IKPUB.Y': y}
    print("Sending message is: ", mes)
    response = requests.put('{}/{}'.format(API_URL, "IKRegReq"), json = mes)		
    print(response.json())

def IKRegVerify(code):
    mes = {'ID':stuID, 'CODE': code}
    print("Sending message is: ", mes)
    response = requests.put('{}/{}'.format(API_URL, "IKRegVerif"), json = mes)
    if((response.ok) == False): raise Exception(response.json())
    else:
        print(response.json())
        f = open('Identity_Key.txt', 'w')
        f.write("IK.Prv: "+str(IKey_Pr)+"\n"+"IK.Pub.x: "+str(IKey_Pub.x)+"\n"+"IK.Pub.y: "+str(IKey_Pub.y))
        f.close()

def SPKReg(h,s,x,y):
    mes = {'ID':stuID, 'H': h, 'S': s, 'SPKPUB.X': x, 'SPKPUB.Y': y}
    print("Sending message is: ", mes)
    response = requests.put('{}/{}'.format(API_URL, "SPKReg"), json = mes)
    print(response.json())			

def OTKReg(keyID,x,y,hmac):
    mes = {'ID':stuID, 'KEYID': keyID, 'OTKI.X': x, 'OTKI.Y': y, 'HMACI': hmac}
    print("Sending message is: ", mes)
    response = requests.put('{}/{}'.format(API_URL, "OTKReg"), json = mes)		
    print(response.json())
    if((response.ok) == False): return False
    else: return True


def ResetIK(rcode):
    mes = {'ID':stuID, 'RCODE': rcode}
    print("Sending message is: ", mes)
    response = requests.delete('{}/{}'.format(API_URL, "ResetIK"), json = mes)		
    print(response.json())
    if((response.ok) == False): return False
    else: return True

def ResetSPK(h,s):
    mes = {'ID':stuID, 'H': h, 'S': s}
    print("Sending message is: ", mes)
    response = requests.delete('{}/{}'.format(API_URL, "ResetSPK"), json = mes)		
    print(response.json())
    if((response.ok) == False): return False
    else: return True


def ResetOTK(h,s):
    mes = {'ID':stuID, 'H': h, 'S': s}
    print("Sending message is: ", mes)
    response = requests.delete('{}/{}'.format(API_URL, "ResetOTK"), json = mes)		
    print(response.json())
        
def concat_integers(a, b):
    binary_a = format(a, 'b') 
    binary_b = format(b, 'b')  
    binary_b = binary_b.zfill((len(binary_b) + 7) // 8 * 8)
    combined_binary = binary_a + binary_b
    combined_integer = int(combined_binary, 2)
    combined_bytes = combined_integer.to_bytes((combined_integer.bit_length() + 7) // 8, byteorder='big')
    return combined_bytes, combined_integer

        
def generateDigest(message):
    hash_obj = SHA3_256.new(message)
    hash_digest = int.from_bytes(hash_obj.digest(), byteorder='big')
    return hash_digest

        
def keyGeneration(n, P):
    sA = random.randint(1, n-2)
    qA = sA * P
    return qA, sA


def create_digital_signature(n,P, m, sA):
    k = random.randint(1, n - 2)
    R = k * P
    r = R.x % n
    h = generateDigest(concat_integers(r,m)[0]) % n
    s = (k - sA * h) % n
    return (h, s)

def create_digital_signature_with_k(n,P, m, sA,k):
    R = k * P
    r = R.x % n
    h = generateDigest(concat_integers(r,m)[0]) % n
    s = (k - sA * h) % n
    return (h, s)

def signatureVerification(s, P, h, qA, n, m):
    V = (s*P) + (h*qA)
    v = V.x % n
    h_prime = generateDigest(concat_integers(v,m)[0]) % n
    return h == h_prime

#Pseudo-client will send you 5 messages to your inbox via server when you call this function
def PseudoSendMsg(h,s):
    mes = {'ID':stuID, 'H': h, 'S': s}
    print("Sending message is: ", mes)
    response = requests.put('{}/{}'.format(API_URL, "PseudoSendMsg"), json = mes)		
    print(response.json())

#Get your messages. server will send 1 message from your inbox
def ReqMsg(h,s):
    mes = {'ID':stuID, 'H': h, 'S': s}
    print("Sending message is: ", mes)
    response = requests.get('{}/{}'.format(API_URL, "ReqMsg"), json = mes)	
    print(response.json())	
    if((response.ok) == True): 
        res = response.json()
        return res["IDB"], res["OTKID"], res["MSGID"], res["MSG"], res["IK.X"], res["IK.Y"], res["EK.X"], res["EK.Y"]

#Get the list of the deleted messages' ids.
def ReqDelMsg(h,s):
    mes = {'ID':stuID, 'H': h, 'S': s}
    print("Sending message is: ", mes)
    response = requests.get('{}/{}'.format(API_URL, "ReqDelMsgs"), json = mes)      
    print(response.json())      
    if((response.ok) == True): 
        res = response.json()
        return res["MSGID"]

#If you decrypted the message, send back the plaintext for checking
def Checker(stuID, stuIDB, msgID, decmsg):
    mes = {'IDA':stuID, 'IDB':stuIDB, 'MSGID': msgID, 'DECMSG': decmsg}
    print("Sending message is: ", mes)
    response = requests.put('{}/{}'.format(API_URL, "Checker"), json = mes)		
    print(response.json())

def encodeParam(x):
    if(type(x) == str):
        return x.encode()
    elif(type(x) == int):
        return x.to_bytes((x.bit_length() + 7) // 8, byteorder="big")
    elif(type(x) == Point):
        return (encodeParam(x.x) + encodeParam(x.y))
    
    
def generateSessionKey(IKB_Pub,SPKA_Pri, EKB_Pub, IKA_Pri, OTKA_Pri):
    T1 = IKB_Pub * SPKA_Pri
    T2 = EKB_Pub * IKA_Pri
    T3 = EKB_Pub * SPKA_Pri
    T4 = EKB_Pub * OTKA_Pri
    U = concat_integers(T1.x,T1.y)[0] +  concat_integers(T2.x,T2.y)[0] +  concat_integers(T3.x,T3.y)[0] +  concat_integers(T4.x,T4.y)[0] + b'WhatsUpDoc'
    k_s = SHA3_256.new(U).digest()
    return k_s,U

def generateKdfChain(k_s):
    kENC = SHA3_256.new(k_s + b'JustKeepSwimming').digest()
    kHMAC = SHA3_256.new(k_s + kENC + b'HakunaMatata').digest()
    KDFNext = SHA3_256.new(kENC + kHMAC + b'OhanaMeansFamily').digest()
    return kENC,kHMAC,KDFNext

############## The new functions of phase 3 ###############

#Pseudo-client will send you 5 messages to your inbox via server when you call this function
def PseudoSendMsgPH3(h,s):
    mes = {'ID': stuID, 'H': h, 'S': s}
    print("Sending message is: ", mes)
    response = requests.put('{}/{}'.format(API_URL, "PseudoSendMsgPH3"), json=mes)
    print(response.json())

# Send a message to client idB
def SendMsg(idA, idB, otkID, msgid, msg, ikx, iky, ekx, eky):
    mes = {"IDA": idA, "IDB": idB, "OTKID": int(otkID), "MSGID": msgid, "MSG": msg, "IK.X": ikx, "IK.Y": iky, "EK.X": ekx, "EK.Y": eky}
    print("Sending message is: ", mes)
    response = requests.put('{}/{}'.format(API_URL, "SendMSG"), json=mes)
    print(response.json())    


# Receive KeyBundle of the client stuIDB
def reqKeyBundle(stuID, stuIDB, h, s):
    key_bundle_msg = {'IDA': stuID, 'IDB':stuIDB, 'S': s, 'H': h}
    print("Requesting party B's Key Bundle ...")
    response = requests.get('{}/{}'.format(API_URL, "ReqKeyBundle"), json=key_bundle_msg)
    print(response.json()) 
    if((response.ok) == True):
        res = response.json()
        return res['KEYID'], res['IK.X'], res['IK.Y'], res['SPK.X'], res['SPK.Y'], res['SPK.H'], res['SPK.s'], res['OTK.X'], res['OTK.Y']
        
    else:
        return -1, 0, 0, 0, 0, 0, 0, 0, 0


#Status control. Returns #of messages and remained OTKs
def Status(stuID, h, s):
    mes = {'ID': stuID, 'H': h, 'S': s}
    print("Sending message is: ", mes)
    response = requests.get('{}/{}'.format(API_URL, "Status"), json=mes)
    print(response.json())
    if (response.ok == True):
        res = response.json()
        return res['numMSG'], res['numOTK'], res['StatusMSG']


############## The new functions of BONUS ###############

# Exchange partial keys with users 2 and 4
def ExchangePartialKeys(stuID, z1x, z1y, h, s):
    request_msg = {'ID': stuID, 'z1.x': z1x, 'z1.y': z1y, 'H': h, 'S': s}
    print("Sending your PK (z) and receiving others ...")
    response = requests.get('{}/{}'.format(API_URL, "ExchangePartialKeys"), json=request_msg)
    if ((response.ok) == True):
        print(response.json())
        res = response.json()
        return res['z2.x'], res['z2.y'], res['z4.x'], res['z4.y']
    else:
        print(response.json())
        return 0, 0, 0, 0


# Exchange partial keys with user 3
def ExchangeXs(stuID, x1x, x1y, h, s):
    request_msg = {'ID': stuID, 'x1.x': x1x, 'x1.y': x1y, 'H': h, 'S': s}
    print("Sending your x and receiving others ...")
    response = requests.get('{}/{}'.format(API_URL, "ExchangeXs"), json=request_msg)
    if ((response.ok) == True):
        print(response.json())
        res = response.json()
        return res['x2.x'], res['x2.y'], res['x3.x'], res['x3.y'], res['x4.x'], res['x4.y']
    else:
        print(response.json())
        return 0, 0, 0, 0, 0, 0

# Check if your conference key is correct
def BonusChecker(stuID, Kx, Ky):
    mes = {'ID': stuID, 'K.x': Kx, 'K.y': Ky}
    print("Sending message is: ", mes)
    response = requests.put('{}/{}'.format(API_URL, "BonusChecker"), json=mes)
    print(response.json())

E = Curve.get_curve('secp256k1')
n = E.order
p = E.field
P = E.generator
a = E.a
b = E.b

IKey_Pub, IKey_Pr = keyGeneration(n, P)

print("Identitiy Key is created")  
print("IKey is a long term key and shouldn't be changed and private part should be kept secret. But this is a sample run, so here is my private IKey:\n" + str(IKey_Pr))
print("My ID number is", stuID,)
print("Converted my ID to bytes in order to sign it:",stuID.to_bytes(2, 'big'))
print("\n+++++++++++++++++++++++++++++++++++++++++++++\n")
print("Signature of my ID number is:")
h, s = create_digital_signature(n, P, stuID, IKey_Pr)
print("h=", h)
print("s=", s)
print("\n+++++++++++++++++++++++++++++++++++++++++++++\n")


print("Sending signature and my IKEY to server via IKRegReq() function in json format")
IKRegReq(h, s, IKey_Pub.x, IKey_Pub.y)
print("Received the verification code through email")
print("\n+++++++++++++++++++++++++++++++++++++++++++++\n")
verificationCode = int(input("Enter verification code which is sent to you: "))
print("\n+++++++++++++++++++++++++++++++++++++++++++++\n")
print("\nSending the verification code to server via IKRegVerify() function in json format")
IKRegVerify(verificationCode)
print("\n+++++++++++++++++++++++++++++++++++++++++++++\n")
print("Generating SPK...")  
SPKPUB, SPKPRI = keyGeneration(n, P)
print("Private SPK:", SPKPRI)
print("Public SPK.x:", SPKPUB.x)
print("Public SPK.y:", SPKPUB.y)
SPKPUBx = SPKPUB.x
SPKPUBy = SPKPUB.y
print("Convert SPK.x and SPK.y to bytes in order to sign them then concatenate them\n Result will be like:", concat_integers(SPKPUBx, SPKPUBy)[0])
print("\n+++++++++++++++++++++++++++++++++++++++++++++\n")

spk_H,spk_S = create_digital_signature(n,P, concat_integers(SPKPUBx,SPKPUBy)[1], IKey_Pr)
print("Signature of SPK is:")
print("h=",spk_H)
print("s=",spk_S)
print("Sending SPK and the signatures to the server via SPKReg() function in json format...")
print("\n+++++++++++++++++++++++++++++++++++++++++++++\n")
SPKReg(spk_H,spk_S,SPKPUB.x, SPKPUB.y)
SPKPUB_server = Point(SPKPUB.x, SPKPUB.y,
                      Curve.get_curve('secp256k1'))
print("\n+++++++++++++++++++++++++++++++++++++++++++++\n")
print("Creating HMAC key (Diffie Hellman)")
T = SPKPRI * IKey_Ser 
print("T is", T)
U = b'TheHMACKeyToSuccess' + concat_integers(T.y, T.x)[0]
print("U is", U)
kHMAC = SHA3_256.new(U).digest()
print("HMAC key is created", kHMAC)
print("\n+++++++++++++++++++++++++++++++++++++++++++++\n")
print("Creating OTKs starting from index 0...")
print("\n+++++++++++++++++++++++++++++++++++++++++++++\n")
OTKRegCount = -1
OTKList = []
for y in range(0, 11): 
    OTKPub, OTKPri = keyGeneration(n, P)
    print("{i}th key generated. Private part={private}".format(
            i=y, private=OTKPri))
    print("Public (x coordinate)={x_coor}".format(x_coor=OTKPub.x))
    print("Public (y coordinate)={y_coor}".format(y_coor=OTKPub.y))
    OTKPublicByte = concat_integers(OTKPub.x,OTKPub.y)[0]
    print("x and y coordinates of the OTK converted to bytes and concatanated\nMessage", OTKPublicByte)
    hmac = HMAC.new(kHMAC, OTKPublicByte, digestmod=SHA256).hexdigest()
    print("HMAC is calculated and converted with 'hexdigest()':", hmac)
    regSuc = OTKReg(y, OTKPub.x, OTKPub.y, hmac)
    if regSuc == True:
        OTKRegCount += 1
        OTKList.append(OTKPri)
    print("\n+++++++++++++++++++++++++++++++++++++++++++++\n")
if OTKRegCount == 10:
    print("OTK keys were generated successfully!")

print("\n+++++++++++++++++++++++++++++++++++++++++++++\n")
print("PHASE 2 FROM NOW ON!")
print("\n+++++++++++++++++++++++++++++++++++++++++++++\n")
print("Checking the inbox for incoming messages")
print("+++++++++++++++++++++++++++++++++++++++++++++\n")
print("Signing my stuID with my private IK\n")
print("In signature generation I fixed the random variable to 1748178 so that you can re-generate if you want\n")
k = 1748178 
h,s = create_digital_signature_with_k(n, P, stuID, IKey_Pr, k)    
PseudoSendMsg(h , s)
print("\n+++++++++++++++++++++++++++++++++++++++++++++\n")
isFirst = True
messagesDict = {}
for i in range(5):
    stuIDB, OTKID, MSGID, MSG, IKX, IKY, EKX, EKY = ReqMsg(h, s)
    print("\nI got this from client {p}:".format(p=stuIDB))
    print()
    print(MSG)
    print("\nConverting message to bytes to decrypt it...\n")
    MSG_withBytes = MSG.to_bytes((MSG.bit_length() + 7) // 8, byteorder="big")
    print("Converted message is:")
    print(MSG_withBytes)
    if(isFirst):
        EKB_Pub = Point(EKX, EKY, Curve.get_curve('secp256k1'))
        IKB_Pub = Point(IKX, IKY, Curve.get_curve('secp256k1'))
        k_s, _ = generateSessionKey(IKB_Pub,SPKPRI, EKB_Pub, IKey_Pr, OTKList[OTKID])
        print("\nGenerating the key Ks, Kenc, & Khmac and then the HMAC value ..")
        kENC,kHMAC,KDFNext = generateKdfChain(k_s)
        isFirst = False
    else:
        print("\nGenerating the key Ks, Kenc, & Khmac and then the HMAC value ..")
        kENC,kHMAC,KDFNext = generateKdfChain(KDFNext)
    
    nonce = MSG_withBytes[:8]
    MAC = MSG_withBytes[-32:]
    cipher = MSG_withBytes[8:-32]
    AES_CTR = AES.new(kENC, AES.MODE_CTR, nonce=nonce)
    hmac = HMAC.new(kHMAC, msg=cipher, digestmod=SHA256).digest()
    print("\nhmac is:", hmac)
    print()
    if(hmac == MAC):
        print("Hmac value is verified")
        plaintext = AES_CTR.decrypt(cipher).decode("UTF-8")
        print("The collected plaintext:", plaintext)
        Checker(stuID, stuIDB, MSGID, plaintext)
        print()
        print("+++++++++++++++++++++++++++++++++++++++++++++\n")
        messagesDict[MSGID] = plaintext
    else:
        print("Hmac value couldn't be verified")
        Checker(stuID, stuIDB, MSGID, "INVALIDHMAC")
        print()
        print("+++++++++++++++++++++++++++++++++++++++++++++\n")
        messagesDict[MSGID] = "INVALIDHMAC"
        

response = ReqDelMsg(h, s)
valid_msgs = {}
print("\nChecking whether there were some deleted messages!! ")
print("==========================================")
for message,value in messagesDict.items():
    if(value != "INVALIDHMAC"):
        if message in response:
            print("Message {} - Was deleted by sender - X".format(message))
        else:
            print(f"Message {message} -", value, "- Read")
            valid_msgs[message] = value

print("Signing the stuIDB of party B with my private IK")
print("In signature generation I fixed the random variable to 1748178 so that you can re-generate if you want)")
print("Requesting party B's OTK ...")
h_B, s_B = create_digital_signature_with_k(n,P, stuIDB, IKey_Pr,k)
KEYID,IKX, IKY, SPKX, SPKY, SPKH, SPKS, OTKX, OTKY = reqKeyBundle(stuID, stuIDB, h_B, s_B)
print("Verifying the server's SPK...")

isVerified = signatureVerification(SPKS, P, SPKH, Point(IKX, IKY, Curve.get_curve('secp256k1')), n, concat_integers(SPKX,SPKY)[1])
print("Is SPK verified?", isVerified)
print("The other party's OTK public key is acquired from the server ...")
print("Requesting messages from the pseudo-client...")
print("Signing my stuID with my private IK")
h,s = create_digital_signature_with_k(n, P, stuID, IKey_Pr, k)    
print("In signature generation I fixed the random variable to 1748178 so that you can re-generate if you want")


isFirst = True
for message,value in valid_msgs.items():
    print("Get a message from the inbox ...")
    print("Generating the KDF chain for the encryption and the MAC value generation")
    print("Generating session key / Phase 3...")
    E_PUB, E_PRI = keyGeneration(n,P)
    if(isFirst):
        k_s, U = generateSessionKey(IKey_Pr, Point(SPKX, SPKY, Curve.get_curve('secp256k1')), E_PRI, Point(IKX, IKY, Curve.get_curve('secp256k1')), Point(OTKX, OTKY, Curve.get_curve('secp256k1')))
        kENC,kHMAC,KDFNext = generateKdfChain(k_s)
        isFirst = False
    else:
        kENC,kHMAC,KDFNext = generateKdfChain(KDFNext)
        
    print("U is:", U)
    MSG_withBytes = value.encode()
    AES_CTR = AES.new(kENC, AES.MODE_CTR)
    nonce = AES_CTR.nonce
    cipher = AES_CTR.encrypt(MSG_withBytes)
    hmac = HMAC.new(kHMAC, msg=cipher, digestmod=SHA256).digest()
    MSG_int = int.from_bytes(nonce + cipher + hmac, byteorder='big')
    print("Sending the message to the server, so it would deliver it to pseudo-client/user whenever it is active ...")
    SendMsg(stuID, stuIDB, KEYID, message, MSG_int, IKey_Pub.x, IKey_Pub.y, E_PUB.x, E_PUB.y)
    

print("\n########## BONUS ###################\n")
print("Generating my partial conference key")
z1, r1 = keyGeneration(n,P)
print("Signing my pratial conference key")
h_, s_ = create_digital_signature(n,P, concat_integers(z1.x, z1.y)[1], IKey_Pr)
z2x, z2y, z4x, z4y = ExchangePartialKeys(stuID, z1.x, z1.y, h_, s_)

print("Exchanging partial keys")
z4 = Point(z4x, z4y, Curve.get_curve('secp256k1'))
z2 = Point(z2x, z2y, Curve.get_curve('secp256k1'))

x1 = r1 * z2 - r1 * z4

print("\ncalculated x1:", x1)

h_, s_ = create_digital_signature(n,P, concat_integers(x1.x, x1.y)[1], IKey_Pr)
x2x, x2y, x3x, x3y, x4x, x4y = ExchangeXs(stuID, x1.x, x1.y, h_, s_)

x2 = Point(x2x, x2y, Curve.get_curve('secp256k1'))
x3 = Point(x3x, x3y, Curve.get_curve('secp256k1'))
x4 = Point(x4x, x4y, Curve.get_curve('secp256k1'))

K = r1 * (z4*4) + x1 * 3 + x2 * 2 + x3 * 1

BonusChecker(stuID, K.x, K.y)

print("\n")
print("Checking the status of the inbox ...")
numMsg, numOTK, statusmes = Status(stuID, h, s)

if(numOTK < 3):
    print("You have", numMsg, "unread messages in your mailbox.")
    print("You have only", numOTK ,"OTK left. Please register new OTKs")
    ResetOTK(h, s)
    T = SPKPRI * IKey_Ser 
    U = b'TheHMACKeyToSuccess' + concat_integers(T.y, T.x)[0]
    kHMAC = SHA3_256.new(U).digest()
    newOTKlist = []
    y = 0
    while y < 10: 
        OTKPub, OTKPri = keyGeneration(n, P)
        OTKPublicByte = concat_integers(OTKPub.x,OTKPub.y)[0]
        hmac = HMAC.new(kHMAC, OTKPublicByte, digestmod=SHA256).hexdigest()
        regSuc = OTKReg(y, OTKPub.x, OTKPub.y, hmac)
        if regSuc == True:
            y += 1
            newOTKlist.append(OTKPri)
    OTKList = newOTKlist

else:
    print("You have", numMsg, "unread messages in your mailbox.")
    print("You have", numOTK, "OTKs left.")
    
print("\n+++++++++++++++++++++++++++++++++++++++++++++\n")
print("Trying to delete OTKs...") 
ResetOTK(h, s)
print("\n+++++++++++++++++++++++++++++++++++++++++++++\n")
print("Trying to delete OTKs but sending wrong signatures...") 
ResetOTK(s, h)
print("\n+++++++++++++++++++++++++++++++++++++++++++++\n")
print("Trying to delete SPKs...")
ResetSPK(h, s)
print("\n+++++++++++++++++++++++++++++++++++++++++++++\n")
print("Trying to delete Identity Key...")
rcode = int( input("Please enter your rcode (reset code) that was sent via email: "))
ResetIK(rcode)








Identitiy Key is created
IKey is a long term key and shouldn't be changed and private part should be kept secret. But this is a sample run, so here is my private IKey:
82705712224810244536979025993970985919240767091615862348425776987263299324576
My ID number is 28315
Converted my ID to bytes in order to sign it: b'n\x9b'

+++++++++++++++++++++++++++++++++++++++++++++

Signature of my ID number is:
h= 92393044051842753277565546721820903198567889991922687052171503031231321895770
s= 111058496225889518418324201405947885852335639890399755744077225758674169286177

+++++++++++++++++++++++++++++++++++++++++++++

Sending signature and my IKEY to server via IKRegReq() function in json format
Sending message is:  {'ID': 28315, 'H': 92393044051842753277565546721820903198567889991922687052171503031231321895770, 'S': 111058496225889518418324201405947885852335639890399755744077225758674169286177, 'IKPUB.X': 47179142778739536527209209744411766517286934739026976258912926118669910691753, 'IKPUB.Y': 2323

OTK with ID number 5 is registered successfully

+++++++++++++++++++++++++++++++++++++++++++++

6th key generated. Private part=89550205323979961598582369966416018703310736462735484908592671568893185769713
Public (x coordinate)=12333107643583416111518904306196486766987493245290663039921381532617485026860
Public (y coordinate)=63248629399880878998017435588345369310814618968428440984005056435770377062495
x and y coordinates of the OTK converted to bytes and concatanated
Message b'\x1bDJ\xa3Id\xf2I\x1f\xd0\x82\x1b\xa4\xef1fL-\xbe\xe3\x8a\xe4\np3\xe2\xb0\xeb\x0f\x8f\x86,\x8b\xd5t\xa3\xed\x06\x18w\xd4\x00\xce\xb2\x0eE@W\xd51\x8e\xa1lN\x16p\xd0\xf9\xcc!Q\xe7H_'
HMAC is calculated and converted with 'hexdigest()': f1d0209a4cb32358d1d405b24f64b362348f40b6e9c02d44eec7d8df49e87bac
Sending message is:  {'ID': 28315, 'KEYID': 6, 'OTKI.X': 12333107643583416111518904306196486766987493245290663039921381532617485026860, 'OTKI.Y': 632486293998808789980174355883453693108146189684284409840050564357703770

{'KEYID': 1146, 'IK.X': 68661839483667325786272306987786394606033305348910618042150907110486525049700, 'IK.Y': 44964402825465717795087681792697928350125994486372414706032731370922079638308, 'SPK.X': 105874942176046182215855241144799728283463100749238818368454064789666974635508, 'SPK.Y': 7959031208481643445881131181934563759805545121515936359453003609666427670067, 'SPK.H': 104596165350335479171431889584729050578191755091185810227294476593899788798151, 'SPK.s': 76880615626451781902613364462811401337730164445434158620279702513445265413640, 'OTK.X': 18432336695647843527335593376175157580952973611508288511819151716601576104202, 'OTK.Y': 19422096905159881835172121408755499335375389248063431543987836799310504395504}
Verifying the server's SPK...
Is SPK verified? True
The other party's OTK public key is acquired from the server ...
Requesting messages from the pseudo-client...
Signing my stuID with my private IK
In signature generation I fixed the random variable to 1748178 so that you can re-

Please enter your rcode (reset code) that was sent via email: 556493
Sending message is:  {'ID': 28315, 'RCODE': 556493}
Deleted successfully


True